In [1]:
from IPython.display import display
import ipywidgets as widgets


In [2]:
def caesar_cipher_encrypt(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            ascii_offset = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - ascii_offset + shift) % 26 + ascii_offset)
        else:
            result += char
    return result

def caesar_cipher_decrypt(text, shift):
    return caesar_cipher_encrypt(text, -shift)

In [3]:
def prepare_text(text):
    clean_text = "".join(char.upper() for char in text if char.isalpha())
    return clean_text

def build_monoalphabetic_mapping(key):
    key = prepare_text(key)
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    mapping = {}

    for i in range(26):
        mapping[alphabet[i]] = key[i] if i < len(key) else alphabet[i]
    return mapping

def monoalphabetic_cipher_encrypt(text, key):
    text = prepare_text(text)
    mapping = build_monoalphabetic_mapping(key)

    result = ""

    for char in text:
        result += mapping[char]
    return result

def monoalphabetic_cipher_decrypt(text, key):
    text = prepare_text(text)
    mapping = build_monoalphabetic_mapping(key)

    result = ""

    for char in text:
        for original, mapped in mapping.items():
            if mapped == char:
                result += original
                break
    return result

In [4]:

def playfair_cipher(plaintext, key, mode):
    alphabet = 'abcdefghiklmnopqrstuvwxyz'
    key = key.lower().replace(' ', '').replace('j', 'i')
    key_square = ''
    for letter in key + alphabet:
        if letter not in key_square:
            key_square += letter

    plaintext = plaintext.lower().replace(' ', '').replace('j', 'i')
    if len(plaintext) % 2 == 1:
        plaintext += 'x'
    digraphs = [plaintext[i:i + 2] for i in range(0, len(plaintext), 2)]

    def encrypt(digraph):
        a, b = digraph
        row_a, col_a = divmod(key_square.index(a), 5)
        row_b, col_b = divmod(key_square.index(b), 5)
        if row_a == row_b:
            col_a = (col_a + 1) % 5
            col_b = (col_b + 1) % 5
        elif col_a == col_b:
            row_a = (row_a + 1) % 5
            row_b = (row_b + 1) % 5
        else:
            col_a, col_b = col_b, col_a
        return key_square[row_a * 5 + col_a] + key_square[row_b * 5 + col_b]

    def decrypt(digraph):
        a, b = digraph
        row_a, col_a = divmod(key_square.index(a), 5)
        row_b, col_b = divmod(key_square.index(b), 5)
        if row_a == row_b:
            col_a = (col_a - 1) % 5
            col_b = (col_b - 1) % 5
        elif col_a == col_b:
            row_a = (row_a - 1) % 5
            row_b = (row_b - 1) % 5
        else:
            col_a, col_b = col_b, col_a
        return key_square[row_a * 5 + col_a] + key_square[row_b * 5 + col_b]

    result = ''
    for digraph in digraphs:
        if mode == 'encrypt':
            result += encrypt(digraph)
        elif mode == 'decrypt':
            result += decrypt(digraph)
    return result

In [5]:
def encrypt_or_decrypt(button):
    plaintext = plaintext_entry.value
    cipher_type = cipher_dropdown.value
    key = key_entry.value
    action = action_dropdown.value

    if cipher_type == "Caesar":
        try:
            key = int(key)
        except ValueError:
            result_label.value = "Invalid key. Please enter a valid integer for the Caesar Cipher."
            return

        if action == "Encrypt":
            result = caesar_cipher_encrypt(plaintext, key)
        else:
            result = caesar_cipher_decrypt(plaintext, key)

    elif cipher_type == "Playfair":
        if not key or not key.isalpha():
            result_label.value = "Invalid key. Please enter a non-empty alphabetical key for the Playfair Cipher."
            return

        result = playfair_cipher(plaintext, key, action.lower())

    elif cipher_type == "Monoalphabetic":
        if action == "Encrypt":
            result = monoalphabetic_cipher_encrypt(plaintext, key)
        else:
            result = monoalphabetic_cipher_decrypt(plaintext, key)

    else:
        result = "Invalid cipher type"

    result_label.value = f"<b>{action}ion:</b> {result}"

In [ ]:
# Create input widgets
plaintext_entry = widgets.Text(description="Plaintext:", style={'description_width': 'initial'})
cipher_dropdown = widgets.Dropdown(options=["Caesar", "Playfair", "Monoalphabetic"], description="Cipher:", style={'description_width': 'initial'})
key_entry = widgets.Text(description="Key:", style={'description_width': 'initial'})
action_dropdown = widgets.Dropdown(options=["Encrypt", "Decrypt"], description="Action:", style={'description_width': 'initial'})
encrypt_button = widgets.Button(description="Encrypt/Decrypt", button_style='success')
clear_button = widgets.Button(description="Clear", button_style='danger')
result_label = widgets.HTML()

# Set up event handlers
encrypt_button.on_click(encrypt_or_decrypt)
clear_button.on_click(lambda x: clear_widgets())

# Display widgets
def clear_widgets():
    for widget in [plaintext_entry, key_entry, result_label]:
        widget.value = ""

display(widgets.HBox([widgets.VBox([plaintext_entry, cipher_dropdown, key_entry, action_dropdown]), encrypt_button, clear_button, result_label]))